In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Aniridia"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Aniridia/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Aniridia/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Aniridia/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Aniridia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Step 1: Look for directories related to Aniridia (a congenital eye disorder)
tcga_subdirs = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {tcga_subdirs}")

# Check if any directories contain relevant terms to Aniridia (eye-related)
aniridia_related_terms = ["eye", "ocular", "iris", "aniridia", "ophthalmologic", "uveal"]
potential_dirs = []

for directory in tcga_subdirs:
    if any(term.lower() in directory.lower() for term in aniridia_related_terms):
        potential_dirs.append(directory)

print(f"Potential {trait}-related directories found: {potential_dirs}")

if potential_dirs:
    # Select the most specific match - TCGA_Ocular_melanomas_(UVM) is related to eye disorders
    target_dir = potential_dirs[0]
    target_path = os.path.join(tcga_root_dir, target_dir)
    
    print(f"Selected directory: {target_dir}")
    
    # Get the clinical and genetic data file paths
    clinical_path, genetic_path = tcga_get_relevant_filepaths(target_path)
    
    # Load the datasets
    clinical_df = pd.read_csv(clinical_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_path, sep='\t', index_col=0)
    
    # Print column names of clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Check if we have both gene data and potential trait data
    has_gene_data = not genetic_df.empty
    has_potential_trait_data = not clinical_df.empty
    
    # Record our initial assessment
    validate_and_save_cohort_info(
        is_final=False, 
        cohort="TCGA", 
        info_path=json_path, 
        is_gene_available=has_gene_data, 
        is_trait_available=has_potential_trait_data
    )
else:
    print(f"No TCGA subdirectory contains terms directly related to {trait}.")
    print("TCGA is primarily a cancer genomics database and may not have specific data for this condition.")
    
    # Marking the trait as unavailable in the cohort_info.json
    validate_and_save_cohort_info(
        is_final=False, 
        cohort="TCGA", 
        info_path=json_path, 
        is_gene_available=False, 
        is_trait_available=False
    )
    
    print(f"Task completed: {trait} data not available in TCGA dataset.")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 'cytogenetic_abnormality', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'extranocular_nodule_size', 'extrascleral_extension', 'extravascular_matrix_patterns', 'eye_color', 'form_completion_date', 'gender', 'gene_expression_profile', 'height', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_pathologic_diagnosis_method', 'initial_weight', 'is_ffpe', 'lost_follow_up', 'metastatic_site', 'mitotic_count', 'new_neoplasm_event_occurrence_anatom

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]
candidate_gender_cols = ["gender"]

# Let's access the clinical data
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Ocular_melanomas_(UVM)")
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

# Extract and preview the candidate columns for age
age_preview = {}
if candidate_age_cols:
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()
    print("Age columns preview:")
    print(age_preview)

# Extract and preview the candidate columns for gender
gender_preview = {}
if candidate_gender_cols:
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [47, 56, 54, 51, 76], 'days_to_birth': [-17514, -20539, -19894, -18948, -28025]}
Gender columns preview:
{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Analyze the candidate columns for age and gender information

# For age, we have two options: 'age_at_initial_pathologic_diagnosis' and 'days_to_birth'
# 'age_at_initial_pathologic_diagnosis' has direct age values (in years)
# 'days_to_birth' has negative values representing days before birth (need conversion)
# Let's choose 'age_at_initial_pathologic_diagnosis' as it's more straightforward
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender, we only have one candidate column: 'gender'
# The values look appropriate ('MALE', 'FEMALE')
gender_col = 'gender'

# Print the selected columns with their preview values from the previous step output
print(f"Selected age column: {age_col}")
print(f"Selected age values preview: {[47, 56, 54, 51, 76]}")
print(f"Selected gender column: {gender_col}")
print(f"Selected gender values preview: {['FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected age values preview: [47, 56, 54, 51, 76]
Selected gender column: gender
Selected gender values preview: ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize the clinical features
# Get file paths using the selected ocular melanoma dataset from Step 1
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Ocular_melanomas_(UVM)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Create standardized clinical features dataframe with trait, age, and gender
# Using tumor/normal classification as the proxy for Aniridia-related trait
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using predefined trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")
print(clinical_features.head())

# Step 2: Normalize gene symbols in gene expression data
# Transpose the genetic data to have genes as rows
genetic_data = genetic_df.copy()

# Normalize gene symbols using the NCBI Gene database synonyms
normalized_gene_data = normalize_gene_symbols_in_index(genetic_data)

# Save normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to get samples as rows, genes as columns
genetic_data_transposed = normalized_gene_data.T

# Ensure clinical and genetic data have the same samples (index values)
common_samples = clinical_features.index.intersection(genetic_data_transposed.index)
clinical_subset = clinical_features.loc[common_samples]
genetic_subset = genetic_data_transposed.loc[common_samples]

# Combine clinical and genetic data
linked_data = pd.concat([clinical_subset, genetic_subset], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values - linked data shape: {linked_data.shape}")

# Step 5: Determine biased features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)
print(f"After removing biased features - linked data shape: {linked_data.shape}")

# Step 6: Validate data quality and save cohort info
# First check if we have both gene and trait data
is_gene_available = linked_data.shape[1] > 3  # More than just trait, Age, Gender
is_trait_available = trait in linked_data.columns

# Take notes of special findings
notes = f"TCGA Ocular Melanomas dataset processed. Used tumor/normal classification as a proxy for {trait} analysis."

# Validate the data quality
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=notes
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality concerns")

Clinical data saved to ../../output/preprocess/Aniridia/clinical_data/TCGA.csv
Clinical data shape: (80, 3)
                 Aniridia  Age  Gender
sampleID                              
TCGA-RZ-AB0B-01         1   47       0
TCGA-V3-A9ZX-01         1   56       1
TCGA-V3-A9ZY-01         1   54       1
TCGA-V4-A9E5-01         1   51       0
TCGA-V4-A9E7-01         1   76       1


Normalized gene data saved to ../../output/preprocess/Aniridia/gene_data/TCGA.csv
Normalized gene data shape: (19848, 80)
Linked data shape: (80, 19851)


After handling missing values - linked data shape: (80, 19851)
Quartiles for 'Aniridia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Aniridia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 61.5
  75%: 74.25
Min: 22
Max: 86
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 35 occurrences. This represents 43.75% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

After removing biased features - linked data shape: (80, 19851)
Linked data not saved due to quality concerns
